In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio.v2 as imageio
import tqdm
import shutil
import os
from IPython.display import Video
plt.rcdefaults()

In [2]:
!pip install tqdm

In [5]:
def animate_images(sequence=None, crops = (700, 300), h = 500, image_dir=None, df_dir = None, file_name = "animation", dir=None):
    if sequence is None:
        IMAGE_DIR = image_dir
        DF_DIR = df_dir
    else:
        sequence = sequence
        IMAGE_DIR = "extracted_images/" + sequence
        DF_DIR = "results/" + sequence + "/centers_using_ap.csv"
    df = pd.read_csv(DF_DIR)
    df["image_id"] = df["id"].apply(lambda x: x.split("/")[-1])
    def show_image(i, save=False, dir=".temp"):
        frame_is_none = False
        image_name = df["image_id"][i]
        image_path = IMAGE_DIR + "/" +image_name
        x = df["x"][i]
        y = df["y"][i]
        if str(x).lower() == "nan":
            frame_is_none = True
        image = plt.imread(image_path)
        c_x = crops[0]
        c_y = crops[1]
        plt.figure(figsize=(1.5*6.08, 6.08))
        if frame_is_none:
            plt.axhline(h//2, c="r")
            plt.axvline(h//2, c="r")
        plt.scatter(x - c_x, y - c_y, c="g", s=50)
        plt.axhline(y-c_y, c="b")
        plt.axvline(x-c_x, c="b")
        plt.axis("off")
        plt.tight_layout()
        plt.imshow(image[c_y : c_y + h, c_x : c_x + h], cmap="gray")
        plt.title(image_name)
        if save:
            if not os.path.exists(dir):
                os.mkdir(dir)
            file_path = dir + "/" + image_name
            plt.savefig(file_path)
            plt.close()
            return file_path
    images = []
    for i in tqdm.tqdm(range(len(df)), desc="Creating Images..."):
        images.append(imageio.imread(show_image(i, save=True)))
    
    if dir is None:
        dir = "."
    print("Creating the video...")
    file_name = dir + "/" + file_name + ".mp4"
    imageio.mimsave(file_name, images)
    print("Deleting the temporary files...")
    shutil.rmtree(".temp")
    return file_name

In [9]:
file = animate_images("ref")

Creating Images...: 100%|██████████| 171/171 [03:07<00:00,  1.10s/it]


Creating the video...


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (656, 437) to (656, 448) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Deleting the temporary files...


In [10]:
Video(file)